In [ ]:
pip install faiss-cpu


In [ ]:
pip install transformers


In [ ]:
import faiss
import numpy as np
from transformers import GPT2Tokenizer, GPT2Model

# Initialize FAISS index (dimension 768 for GPT-2 embeddings)
index = faiss.IndexFlatL2(768)

# Example function to add embeddings to FAISS
def add_embeddings_to_faiss(history):
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2Model.from_pretrained("gpt2")

    for message in history:
        inputs = tokenizer(message, return_tensors="pt")
        embedding = model(**inputs).last_hidden_state.mean(dim=1).detach().numpy()
        index.add(embedding)

# Example function to retrieve relevant history
def retrieve_relevant_history(query):
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2Model.from_pretrained("gpt2")

    # Encode query
    inputs = tokenizer(query, return_tensors="pt")
    query_embedding = model(**inputs).last_hidden_state.mean(dim=1).detach().numpy()

    # Search for the nearest neighbors
    D, I = index.search(query_embedding, k=3)  # k=3 nearest neighbors
    return I


In [ ]:
pip install ipywidgets


In [ ]:
pip install torch


In [ ]:
# Cell 2: Imports and API key variables
import openai
import faiss
import torch
from transformers import GPT2Tokenizer, GPT2Model
import numpy as np

# Set up API keys
OPENAI_API_KEY = 'your key'
openai.api_key = OPENAI_API_KEY


In [ ]:
# Cell 3: Define the conversation history
history = [
    "1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
    "2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
    "3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?",
    "4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.",
    "5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
    "6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?",
    "7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.",
    "8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.",
    "9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.",
    "10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.",
    # Continue for all 20 messages
]


In [ ]:
# Cell 4: Define the function to add embeddings to FAISS
def add_embeddings_to_faiss(history):
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2Model.from_pretrained("gpt2")
    
    # Initialize FAISS index (768 dimensions for GPT-2 embeddings)
    index = faiss.IndexFlatL2(768)
    
    for idx, message in enumerate(history):
        inputs = tokenizer(message, return_tensors="pt")
        with torch.no_grad():
            embedding = model(**inputs).last_hidden_state.mean(dim=1).detach().numpy()
        
        # Add embedding to FAISS index
        index.add(embedding)
    
    return index

# Create FAISS index with history embeddings
faiss_index = add_embeddings_to_faiss(history)


In [ ]:
# Cell 5: Define the RAG mechanism using FAISS
def retrieve_relevant_history(query, faiss_index):
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2Model.from_pretrained("gpt2")

    # Encode the query
    inputs = tokenizer(query, return_tensors="pt")
    with torch.no_grad():
        query_embedding = model(**inputs).last_hidden_state.mean(dim=1).detach().numpy()

    # Query FAISS index for the top 3 most relevant messages
    distances, indices = faiss_index.search(query_embedding, k=3)
    
    # Retrieve the corresponding messages
    relevant_messages = [history[i] for i in indices[0]]
    
    return relevant_messages


In [ ]:
# Cell 6: Function to prepare the prompt
def prepare_prompt(test_prompt, faiss_index):
    # Retrieve relevant history messages using the RAG mechanism
    relevant_messages = retrieve_relevant_history(test_prompt, faiss_index)
    context = " ".join(relevant_messages)
    
    # Combine the retrieved messages with the test prompt
    combined_prompt = f"{context} {test_prompt}"
    
    # Ensure the combined prompt does not exceed the 225 token limit
    if len(combined_prompt.split()) > 225:
        combined_prompt = combined_prompt[:225]
    
    return combined_prompt, relevant_messages


In [ ]:
# Cell 7: Function to test the final prompt
def test_final_prompt():
    final_test_prompt = "Do you think it will help me stay fit?"
    
    # Prepare the prompt by retrieving relevant history
    prepared_prompt, context_referred = prepare_prompt(final_test_prompt, faiss_index)
    
    # Call OpenAI to generate a response
    response = openai.chat.completions.create(
         model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": final_test_prompt}
            ],
            max_tokens=50
    )
    
    # Print the final test prompt, context, and response
    print(f"Final Test Prompt: {final_test_prompt}")
    print(f"Context Referred: {context_referred}")
    print(f"Final Test Prompt Response: {response.choices[0].text.strip()}")

# Call the test function
test_final_prompt()


In [ ]:
import openai

def test_final_prompt_simple():
    final_test_prompt = "Do you think it will help me stay fit?"

    try:
        # Directly create a chat-based completion request
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": final_test_prompt}
            ],
            max_tokens=50
        )
        
        print(f"Final Test Prompt: {final_test_prompt}")
        print(f"Final Test Prompt Response: {response.choices[0].message['content'].strip()}")
    
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the simplified test function
test_final_prompt_simple()


In [ ]:
pip install openai